In [1]:
from contracting.stdlib.bridge.hashing import sha3
import secrets

In [2]:
def rps_contract():
    # This tells the blockchain to make space for data. Variable and Hash are not Python builtins. 
    # They are globals (globals are something you can access without importing it) made available by the contracting. 
    # Calling Variable() creates a new piece of memory within the blockchain, that can hold a single value. 
    # If we say "foo = Variable()" then we can interact with that piece of memory through "foo". 
    # We can set the piece of memory to 1, for example by calling "foo.set(1)".
    # Setting the value will change the value for the next call to the smart contract.
    # We can get the piece of memory by calling "foo.get()".
    next_game_id = Variable()
    
    game_id_to_password_hash = Hash()
    
    game_id_to_player1_choice_hash = Hash()
    game_id_to_player2_choice_hash = Hash()
    
    game_id_to_player1_choice = Hash()
    game_id_to_player2_choice = Hash()
            
    # @construct means that this is the function that will be called ONCE when the smart contract is created
    @construct
    def constructor():
        # Game id starts at 0.
        next_game_id.set(0)
        
    # @export makes this function public, so it can be called by anyone on a deployed contract.
    # By calling start_game player1 creates a game and also submits their hashed and salted choice.
    @export
    def start_game(password_hash, player1_choice_hash):
        # This retrieves unique game Id from the blockchain. 
        unique_game_id = next_game_id.get()
        # This increments the number by 1, so the next game has a unique Id.
        next_game_id.set(next_game_id.get() + 1)
        
        # Hash throws error on integer keys. That's why we convert to a string
        # Remember player1 choice hash, and password hash for this game. 
        game_id_to_password_hash[str(unique_game_id)] = password_hash

        game_id_to_player1_choice_hash[str(unique_game_id)] = player1_choice_hash
        
        return unique_game_id
    
    # By calling submit_choice player2 submits their hashed and salted choice.
    @export
    def submit_choice(game_id, game_password, player2_choice_hash):
        assert_game_exists(game_id)

        # Check that this is the right password for the game.
        assert hashlib.sha3(game_password) == game_id_to_password_hash[str(game_id)], 'Wrong password!'
        # By calling submit_choice player2 reveals the password to everyone. This means that everyone could reach
        # this line of code. We need to ensure that player2's choice can only be submitted once.
        assert game_id_to_player2_choice_hash[str(game_id)] is None, 'Player 2 has already submitted choice.' 
        
        # Remember player2's choice.
        game_id_to_player2_choice_hash[str(game_id)] = player2_choice_hash
        
        return
    
    # Returns 'player1_wins' if player1 is the winner.
    # Returns 'player2_wins' if player2 is the winner.
    # Returns 'tie' if both players made the same choice.
    # Returns 'unfinished' if the game isn't finished

    @export
    def get_outcome(game_id):
        assert_game_exists(game_id)
        
        player1_choice = game_id_to_player1_choice[str(game_id)]
        player2_choice = game_id_to_player2_choice[str(game_id)]
        
        if player1_choice is None or player2_choice is None:
            return "unfinished"
        
        if player1_choice == player2_choice:
            return "tie"

        if beats(player1_choice, player2_choice):
            return "player1_wins"
        
        if beats(player2_choice, player1_choice):
            return "player2_wins"
        
    @export
    def is_valid_choice(choice):
        return choice in ["rock", "paper", "scissors"]
      
    # Returns whether choice1 beats choice2
    @export
    def beats(choice1, choice2):
        assert is_valid_choice(choice1), "choice1 must be a valid choice"
        assert is_valid_choice(choice2), "choice2 must be a valid choice"

        if choice1 == "rock" and choice2 == "scissors":
            return True
        
        if choice1 == "paper" and choice2 == "rock":
            return True
        
        if choice1 == "scissors" and choice2 == "paper":
            return True
        
        return False
    
    # By calling reveal a player can reveal their unhashed choice.
    # Player1 has to call the function with is_player1=true.
    # Player2 has to call the function with is_player1=false.
    @export
    def reveal(game_id, choice, choice_salt, is_player1):
        assert_game_exists(game_id)
        
        # Make sure players can only reveal valid choices
        assert is_valid_choice(choice), "choice must be rock, paper or scissors"
        
        # We don't want the players to be able to reveal before both players have submitted their choices
        assert game_id_to_player1_choice_hash[str(game_id)] is not None, "Player 1 must have submitted their choice"
        assert game_id_to_player2_choice_hash[str(game_id)] is not None, "Player 2 must have submitted their choice"
        
        # Don't allow players to reveal twice
        if is_player1:
            assert game_id_to_player1_choice[str(game_id)] is None, "Player 1 can't reveal their choice twice"
        else:
            assert game_id_to_player2_choice[str(game_id)] is None, "Player 2 can't reveal their choice twice"
        
        # We've made sure that reveal can be called
        
        # Now we need to check that the reveal is correct
        
        salted_choice = choice + choice_salt
        hashed_choice = hashlib.sha3(salted_choice)
        
        if is_player1:
            assert game_id_to_player1_choice_hash[str(game_id)] == hashed_choice, "Player 1 has revealed a choice different from what they submitted"
        else:
            assert game_id_to_player2_choice_hash[str(game_id)] == hashed_choice, "Player 2 has revealed a choice different from what they submitted"
            
        # Now we're sure that the player has revealed the choice they have previously submitted the hashed salted version of
        
        # Remember the choice
        if is_player1:
            game_id_to_player1_choice[str(game_id)] = choice
        else:
            game_id_to_player2_choice[str(game_id)] = choice
        
        return
    
    def assert_game_exists(game_id):
        # Check that this game exists.
        assert game_id < next_game_id.get(), 'Game with id {} does not exist.'.format(game_id)

In [3]:
from contracting.client import ContractingClient

In [4]:
client = ContractingClient(signer='ren')

Clear Redis database so we have a blank slate

In [5]:
client.flush()

In [6]:
client.submit(rps_contract)

Get a handle for the contract that we can interact with

In [7]:
contract = client.get_contract('rps_contract')

Alice (player 1) chooses single use password. Only the person that has the password can join the game and play with Alice. Everything that starts with alice_ is only visible to Alice. 

In [8]:
alice_game_password = "trollbridge"

Alice hashes the password so she can submit it to the blockchain without sharing the actual password. She does this because everything on the blockchain is public, and she wants only the person she chooses to play the game with to have the password. 

*explain hashing

In [9]:
alice_game_password_hash = sha3(alice_game_password)
alice_game_password_hash

'47a5bcfb0d81053f5025ab57e6b94f43751f91bdb16fc0d63595223dc78ec1b4'

Alice chooses rock.

In [10]:
alice_choice = "rock"

We can't submit the choice to the blockchain as plain text, because then Bob (player 2) can see it and win by choosing paper. 

*explain blockchain sequentiality

In [11]:
alice_choice_hash = sha3(alice_choice)
alice_choice_hash

'bd996e2dc82a7c3e2c2da41291648e852b62b01fb09bcb6ed77975c273f08404'

The problem with subiting Alices choice like this is the 3 choices will be hashed the same everytime. Bob (player 2) can know what each of the hashes for the 3 choices and pick paper to win. To fix this Alice needs to pick a random salt to hash with her choice so that Bob can't guess her choice by looking at the hash.

In [12]:
alice_choice_salt = secrets.token_hex(32)
alice_choice_salt

'e537d082c00ca25eebe331eaf5ddd83fa87bb1bebf6dcad7b689b1fe88213afc'

Now we can combine alice_choice and alice_choice_salt and hash them together to create something that Bob can't guess Alices choice from. But Alice can later submit her choice and the salt to prove her choice.

In [13]:
alice_salted_choice = alice_choice + alice_choice_salt
alice_salted_choice

'rocke537d082c00ca25eebe331eaf5ddd83fa87bb1bebf6dcad7b689b1fe88213afc'

In [14]:
alice_salted_choice_hash = sha3(alice_salted_choice)
alice_salted_choice_hash

'687f7855dfedbcf46c8fb3587b4261d7c6d89d1c1ba57f99f128c3474d766832'

Now Alice starts a game so she can invite Bob to play.

In [15]:
alice_game_id = contract.start_game(password_hash=alice_game_password_hash, player1_choice_hash=alice_salted_choice_hash) 
alice_game_id

0

Alice gets back a game Id.

Now Alice has to tell Bob the password and the game Id. This could be done over a messanger or built into the front end of an application. 

Everything that starts with bob_ is only visible to Bob.

In [16]:
bob_game_password = alice_game_password
bob_game_id = alice_game_id

Now it is Bobs turn.
Bob chooses scissors.

In [17]:
bob_choice = "scissors"

And now Bob has to salt his choice.

In [18]:
bob_choice_salt = secrets.token_hex(32)
bob_choice_salt

'66a2200ad9346423180b7e2b36fef0cc41eb7bf0ba6a29dd844da5eb9b2710c5'

And now we combine bob_choice and bob_choice_salt together and hash them.

In [19]:
bob_salted_choice = bob_choice + bob_choice_salt
bob_salted_choice

'scissors66a2200ad9346423180b7e2b36fef0cc41eb7bf0ba6a29dd844da5eb9b2710c5'

In [20]:
bob_salted_choice_hash = sha3(bob_salted_choice)
bob_salted_choice_hash

'2c1d875f5239925d8e455e363f8973ceeb2377c0a7f0adcd1c8beca098658c89'

Bob now needs to submit his choice to the blockchain. Only Bob has the game password so only Bob can join Alices game.

In [21]:
contract.submit_choice(game_password=bob_game_password, game_id=bob_game_id, player2_choice_hash=bob_salted_choice_hash)

Make sure that the things that should not work don't work
TODO check that we can't submit choice for nonexisting game
TODO check that we can't submit choice for game with wrong password
TODO check that we can't submit choice twice

Now that both players have submitted their hashed and salted choices, both players can reveal their choices.
The order doesn't matter. Alice goes first in this example.

In [22]:
contract.reveal(game_id=alice_game_id, choice=alice_choice, choice_salt=alice_choice_salt, is_player1=True)

Bob goes second

In [23]:
contract.reveal(game_id=bob_game_id, choice=bob_choice, choice_salt=bob_choice_salt, is_player1=False)

Now we can see who won the game

In [24]:
contract.get_outcome(game_id=alice_game_id)

'player1_wins'

As expected Alice, who is player1, wins!

## tests

In [30]:
assert contract.is_valid_choice(choice="rock")
assert contract.is_valid_choice(choice="paper")
assert contract.is_valid_choice(choice="scissors")
assert not contract.is_valid_choice(choice="airplane")

In [38]:
assert contract.beats(choice1="rock", choice2="scissors")
assert not contract.beats(choice1="rock", choice2="rock")
assert not contract.beats(choice1="rock", choice2="paper")

assert contract.beats(choice1="paper", choice2="rock")
assert not contract.beats(choice1="paper", choice2="paper")
assert not contract.beats(choice1="paper", choice2="scissors")

assert contract.beats(choice1="scissors", choice2="paper")
assert not contract.beats(choice1="scissors", choice2="scissors")
assert not contract.beats(choice1="scissors", choice2="rock")